In [184]:
import numpy as np
import pandas as pd

In [193]:
df = pd.read_csv("../Data Exploration and Preparation/master_data_with_imputed_budget_and_revenue.csv")

C:\Users\samsu\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (46) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [194]:
df.columns

Index(['movie_id', 'actor_1_gender', 'actor_2_gender', 'actor_3_gender',
       'actor_4_gender', 'actor_5_gender', 'actor_1_name', 'actor_2_name',
       'actor_3_name', 'actor_4_name', 'actor_5_name', 'director_gender',
       'director_name', 'producer_gender', 'producer_name', 'casting_gender',
       'casting_name', 'belongs_to_collection', 'genre_1', 'genre_2',
       'genre_3', 'genre_4', 'production_company_1', 'production_company_2',
       'production_company_3', 'production_country_1', 'production_country_2',
       'production_country_3', 'spoken_language_1', 'spoken_language_2',
       'spoken_language_3', 'adult', 'budget', 'homepage', 'imdb_id',
       'original_language', 'original_title', 'overview', 'popularity',
       'poster_path', 'release_date', 'revenue', 'runtime', 'status',
       'tagline', 'title', 'video', 'vote_average', 'vote_count'],
      dtype='object')

In [195]:
df.drop(['movie_id','actor_1_name','actor_2_name','actor_3_name','actor_4_name','actor_5_name','director_name','producer_name'\
         ,'casting_name','belongs_to_collection','genre_3', 'genre_4','production_company_1','production_company_2','production_company_3'\
         ,'spoken_language_1','spoken_language_2','spoken_language_3','homepage','imdb_id','original_title','overview'\
        ,'poster_path','release_date','runtime','status','tagline','title','video','production_company_1', 'production_company_2',
       'production_company_3','production_country_2', 'production_country_3','original_language'],axis=1,inplace=True)

In [196]:
df.columns

Index(['actor_1_gender', 'actor_2_gender', 'actor_3_gender', 'actor_4_gender',
       'actor_5_gender', 'director_gender', 'producer_gender',
       'casting_gender', 'genre_1', 'genre_2', 'production_country_1', 'adult',
       'budget', 'popularity', 'revenue', 'vote_average', 'vote_count'],
      dtype='object')

Let's only consider the movies for which both budget and revenue information is available.

In [197]:
df=df[(df['budget']!=0)&(df['revenue']!=0)]

drop null rows

In [215]:
df = df[pd.notnull(df['revenue'])]

In [216]:
df.shape

(5654, 18)

Let's define a new column called success, which is 1 when revenue>budget else 0.

In [200]:
df['success'] = [1 if df.iloc[i,-3]>df.iloc[i,-5] else 0 for i in range(df.shape[0])]

In [201]:
df['revenue'] = pd.qcut(df['revenue'],4,labels=False)

In [202]:
df['budget'] = pd.qcut(df['budget'],4,labels=False)

In [204]:
df['vote_count'] = pd.qcut(df['vote_count'],4,labels=False)

In [205]:
df['vote_average'] = pd.qcut(df['vote_average'],4,labels=False)

In [206]:
df['popularity'] = pd.qcut(df['popularity'],4,labels=False)

In [221]:
df[['actor_1_gender','actor_2_gender','actor_3_gender','actor_4_gender','actor_5_gender','director_gender','producer_gender',\
    'casting_gender','budget','revenue']] = df[['actor_1_gender','actor_2_gender','actor_3_gender','actor_4_gender','actor_5_gender','director_gender','producer_gender',\
    'casting_gender','budget','revenue']].apply(pd.to_numeric)

In [226]:
df.drop(['genre_1','genre_2','production_country_1','adult'],axis=1,inplace=True)

In [227]:
df.head()

,actor_1_gender,actor_2_gender,actor_3_gender,actor_4_gender,actor_5_gender,director_gender,producer_gender,casting_gender,budget,popularity,revenue,vote_average,vote_count,success
0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,NaN,2.0,3,3.0,3,3,1
1,2.0,2.0,1.0,0.0,1.0,2.0,2.0,0.0,3.0,3,3.0,3,3,1
2,2.0,2.0,1.0,1.0,1.0,2.0,NaN,NaN,2.0,3,2.0,2,1,1
3,1.0,1.0,1.0,1.0,2.0,2.0,0.0,NaN,1.0,0,2.0,1,0,1
4,2.0,1.0,2.0,1.0,2.0,2.0,1.0,NaN,1.0,2,0.0,1,2,0


In [218]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [228]:
frequent_itemsets = apriori(df, min_support=0.07, use_colnames=True)

C:\Users\samsu\Anaconda3\lib\site-packages\mlxtend\frequent_patterns\apriori.py:51: RuntimeWarning: invalid value encountered in greater_equal
  support_dict = {1: support[support >= min_support]}
C:\Users\samsu\Anaconda3\lib\site-packages\mlxtend\frequent_patterns\apriori.py:52: RuntimeWarning: invalid value encountered in greater_equal
  itemset_dict = {1: ary_col_idx[support >= min_support].reshape(-1, 1)}


In [235]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

In [242]:
rules.sort_values(by=['lift'],ascending=False).head(5)

,antecedants,consequents,support,confidence,lift
180,"(popularity, budget, vote_count)","(success, revenue)",0.091970,0.959615,5.366632
185,"(success, revenue)","(popularity, budget, vote_count)",0.178811,0.493571,5.366632
181,"(popularity, vote_count, success)","(budget, revenue)",0.123099,0.716954,5.060747
184,"(budget, revenue)","(popularity, vote_count, success)",0.141670,0.622971,5.060747
187,"(popularity, budget)","(vote_count, success, revenue)",0.147683,0.597605,5.043071


The rule that high popularity points, combined with budget and vote_count leads to success and high revenue, seems intuitive.

In [241]:
rules.to_csv("rules.csv",encoding='utf8',index=False)